In [1]:
import findspark
findspark.init()
#1a01e0e77d803e8e292dbb639aa77646d41947892fe6f227

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://1c173c8c3406:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 01:49:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from grandeInformacion.spark_utils import SparkUtils

SparkUtils.logs('/home/jovyan/notebooks/data/logs_output')

In [ ]:
from team_name.spark_utils import SparkUtils

students_schema = SparkUtils.generate_schema([("name", "string")])

students_df = spark \
                .readStream \
                .schema(students_schema) \
                .format("json") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/archive") \
                .load("/home/jovyan/notebooks/data/strucutred_streaming_files/")

In [ ]:
query = students_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='20 seconds') \
                .format("console") \
                .start()

query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
sc.stop()